In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --user opencv-contrib-python

In [1]:
#Part 1
import cv2
import numpy as np

#haar_classifier 사용
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')


def face_extractor(img):

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)   #grayscale 사용

    if faces is():
        return None

    for(x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

cap = cv2.VideoCapture(0)
count = 0


while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count+=1
        face = cv2.resize(face_extractor(frame),(200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = './faces/user'+str(count)+'.jpg'
        cv2.imwrite(file_name_path,face)

        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face Cropper',face)
    else:
        print("Face not Found")
        pass

    if cv2.waitKey(1)==13 or count==100:   #사진 100장 저장
        break

cap.release()
cv2.destroyAllWindows()
print('Colleting Samples Complete!!!')

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-470d9ae88d2e>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is():


Face not Found
Face not Found
Face not Found
Face not Found
Face not Found
Face not Found
Face not Found
Face not Found
Face not Found
Colleting Samples Complete!!!


얼굴 샘플 파일 100개 저장

In [9]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
#Part 2
data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]
Training_Data, Labels = [], []
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is None:
        continue    
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
if len(Labels) == 0:
    print("There is no data to train.")
    exit()
Labels = np.asarray(Labels, dtype=np.int32)
model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Training Complete!!!!!")
#모델 학습

#haar 사용
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size = 0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    if faces is():
        return img,[]
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))
    return img,roi   #검출된 좌표에 사각 박스 그림. img에서 검출된 부위를 잘라(roi) 전달

#웹캠 열기 
cap = cv2.VideoCapture(0)

while True:
    #카메라로 부터 사진 한장 읽기 
    ret, frame = cap.read()
    # 얼굴 검출 시도 
    image, face = face_detector(frame)
    try:
        #검출된 사진을 흑백으로 변환 
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        #위에서 학습한 모델로 예측시도
        result = model.predict(face)
        #result[1]은 신뢰도. 0에 가까울수록 자신과 같다는 뜻
        if result[1] < 500:
            confidence = int(100*(1-(result[1])/300))
            # 유사도 화면에 표시 
            display_string = str(confidence)+'% Confidence it is user'
        cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
        #75 보다 크면 동일 인물로 간주해 출석체크
        if confidence > 75:
            cv2.putText(image, "Attendance", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Face Cropper', image)
        else:
           #75 이하면 타인이므로 출석체크X
            cv2.putText(image, "not attend", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Cropper', image)
    except:
        #얼굴 검출 안됐을때
        cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Face Cropper', image)
        pass
    k = cv2.waitKey(1) & 0xff            # 비디오 종료 시 'ESC' 누르기 
    if k == 27:
        break
    #if cv2.waitKey(1)==13:
     #   break
cap.release()
cv2.destroyAllWindows()

<>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?


Model Training Complete!!!!!


<ipython-input-9-c04dbc6f7954>:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is():


# 다른방법

In [11]:
import cv2

# 카메라 호출, USB 카메라를 따로 사용하여 디바이스 순번 ID 1번  사용, 
# 기본 내장 카메라 는 보통 순번 ID 0 번 사용
vcap = cv2.VideoCapture(0) 

# 카메라 속성 설정
vcap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # 세로 사이즈
vcap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # 가로 사이즈

# 얼굴 인식 / 검출 관련 모델(가중치) 파일
# [https://github.com/opencv/opencv/tree/master/data/haarcascades](https://github.com/opencv/opencv/tree/master/data/haarcascades)
casecade_xml = './haarcascade_frontalface_default.xml'

# 모델 파일 불러오기
face_cascade = cv2.CascadeClassifier(casecade_xml)

# 실시간 영상에서 얼굴 검출
def videoDetector(vcap, face_cascade) :
    faces_cnt = 0 # 검출 얼굴수
    # 카메라의 프레임을 지속적으로 받아오기
    while True :
        # vcap.read() 프레임 읽기 
        # ret 은 카메라 상태 이며, 정상 : True, 비정상 : False
        # frame 은 현재시점의 플레임 
        ret, frame = vcap.read()

        # #  숫자키 1 입력시 좌우 대칭 변경
        # if cv2.waitKey(1) == 49 :
        #     # 대칭 처리 
        #     # value < 0 상하좌우 대칭
        #     # value = 0 상하 대칭
        #     # value > 0  좌우 대칭
        #     frame = cv2.flip(frame, 1)
        frame = cv2.flip(frame, 1) # 좌우 대칭 변경

        # 영상 이미지를 그레이스케일로 이진화(검은색, 흰색{True, False})
        #  cv2.cvtColor(대상 이미지, 그레이 스케일)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 영상 이미지에서 얼굴 검출 하기
        #  gray                    # 대상 이미지 행렬
        # ,scaleFactor  = 1.1      # 이미지 피라미드 규모인자 크기(Scale Factor)
        # ,minNeighbors = 5        # 최종 검출영역 확정용 이웃 사각형의 갯수 설정
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)

        # 화면에 검출 된 얼굴 수 가 변경될때마다 출력
        if faces_cnt != len(faces) :
            faces_cnt = len(faces)
            if faces_cnt != 0 :
                print("현재 검출된 얼굴 수 : ", str(faces_cnt))

        # 검출된 안면에 사각형 그리기
        # cv2.rectangle(영상이미지, (x1, y1), (x2, y2), (B, G, R), 두깨, 선형타입)
        # (X1, Y1) 좌측 상단 모서리, (X2, Y2) 우측 하단 모서리.
        if len(faces) :
            for  x, y, w, h in faces :
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255,255,255), 2, cv2.LINE_4)

        # 윈도우창 (Title , 프레임 이미지)
        cv2.imshow("VideoFrame", frame)

        # cvs2.waitKey(1) 1은 밀리세컨으로 키입력값 대기 지연시간이다. ESC로 멈춤
        if cv2.waitKey(1) == 27 :
            vcap.release()          # 메모리 해제 
            cv2.destroyAllWindows() # 모든창 제거, 특정 창만듣을 경우 ("VideoFrame")
            break;                  


# 실시간 영상에서 얼굴 검출 호출
videoDetector(vcap, face_cascade)

현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  3
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 :  2
현재 검출된 얼굴 수 :  1
현재 검출된 얼굴 수 : 

KeyboardInterrupt: 

얼굴만 검출됨. 이름 안나옴

In [7]:
!pip install --user opencv-contrib-python
!pip install dlib
!pip install face_recognition
!pip install flask

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\ADmin\\anaconda3\\Lib\\site-packages\\cv2\\cv2.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Running setup.py clean for dlib
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\ADmin\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ADmin\\AppData\\Local\\Temp\\pip-install-eh3sv7qc\\dlib\\setup.py'"'"'; __file__='"'"'C:\\Users\\ADmin\\AppData\\Local\\Temp\\pip-install-eh3sv7qc\\dlib\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ADmin\AppData\Local\Temp\pip-wheel-cxrxbi4o'
       cwd: C:\Users\ADmin\AppData\Local\Temp\pip-install-eh3sv7qc\dlib\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular file)
  running build_ext
  
  ERROR: CMake must be installed to build dlib
  
  ----------------------------------------
  ERROR: Failed building wheel for dlib
    ERROR: Command errored out

  Using cached dlib-19.22.0.tar.gz (7.4 MB)
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566176 sha256=831c54595b2204ecbbdca19c68ae938d321e6408938b0c4ab7041701706a9bbc
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\b4\4b\8f\751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
  Running setup.py clean for dlib
Successfully built face-recognition-models
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\ADmin\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ADmin\\AppData\\Local\\Temp\\pip-install-iohjv516\\dlib\\setup.py'"'"'; __file__='"'"'C:\\Users\\ADmin\\AppData\\Local\\Temp\\pip-install-iohjv516\\dlib\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ADmin\AppData\Local\Temp\pip-wheel-4re40690'
       cwd: C:\Users\ADmin\AppData\Local\Temp\pip-install-iohjv516\dlib\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\python\dlib\__init__.py' not found (or not a regular file)
  running build_ext
  
  ERROR: CMake must be installed to build dlib
  
  ----------------------------------------
  ERROR: Failed building wheel for dlib
    ERROR: Command errored out

In [4]:
pip install dlib

  Using cached dlib-19.22.0.tar.gz (7.4 MB)
  Created wheel for dlib: filename=dlib-19.22.0-cp38-cp38-win_amd64.whl size=3017946 sha256=179e9fe8272984dc89c592a7c7f079fbd675d92037e71c7790a1dbf5393122e6
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\80\93\82\7b6b3974475c898a823b58a6c0b87013f0782095d43b910e64
Successfully built dlib
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install CMake